In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50,InceptionResNetV2
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras import optimizers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
base_model = InceptionResNetV2(weights = 'imagenet',include_top=False,input_shape=(224,224,3))
#base_model.summary()

In [4]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(2, activation='softmax'))

In [5]:
for layer in base_model.layers: 
  layer.trainable = False
model.compile(optimizer=optimizers.SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

In [6]:
model.load_weights("newrust.h5")

In [8]:
model.save('kerasmodel.pkl')

In [11]:
%%writefile script.py

import numpy as np
import os
import sys
import keras as K
import json
import base64
from io import BytesIO
from PIL import Image
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input

def init():
    
    global model  

    print("Executing init() method...")
    print("Python version: " + str(sys.version) + ", keras version: " + K.__version__)
    # Load the model 
    model = K.models.load_model('azureml-models/kerasmodel/8/kerasmodel.pkl')
    #model = K.models.load_model('kerasmodel.pkl')
    
    return

def predict(model, img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict_classes(x)
    return preds[0]

def run(inputString):
    
    responses = []
    base64Dict = json.loads(inputString)

    for k, v in base64Dict.items():
        img_file_name, base64Img = k, v
    decoded_img = base64.b64decode(base64Img)
    img_buffer = BytesIO(decoded_img)
    #imageData = Image.open(img_buffer).convert("RGB")


#     # Evaluate the model using the input data
#     img = ImageOps.fit(imageData, (32, 32), Image.ANTIALIAS)
#     img_conv = np.array(img) # shape: (32, 32, 3)
#     # Scale pixel intensity
#     x_test = img_conv / 255.0
#     # Reshape
#     x_test = np.moveaxis(x_test, -1, 0)
#     x_test = np.expand_dims(x_test, 0)  # shape (1, 3, 32, 32)

#     y_pred = model.predict(x_test)
#     y_pred = np.argmax(y_pred, axis=-1)
#     # print(y_pred)
#     data = json.loads(inputString)['data']
    #img_input = base64.decodestring(json.dumps(inputString)[0])
    img = image.load_img(img_buffer,target_size=(224, 224))
    #img = json.loads(inputString)['data']
    preds = predict(model, img)
    LABELS = ["Non Rust", "Rust"]
    resp = LABELS[preds]
    responses.append(resp)
    return json.dumps(responses)
    
  
if __name__ == "__main__":
    init()
    # input data
    img_path = '00000668.JPG'
    #input_data = "{\"data\": [" + str(list(img_path)) + "]}"
    encoded = None
    with open(img_path, 'rb') as file:
        encoded = base64.b64encode(file.read())
    img_dict = {img_path: encoded.decode('utf-8')}
    body = json.dumps(img_dict)
    resp = run(body)
    print(resp)

Overwriting script.py


In [10]:
!python script.py

Using TensorFlow backend.
Executing init() method...
Python version: 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 17:14:51) 
[GCC 7.2.0], keras version: 2.1.5
2018-12-26 21:29:34.245122: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-12-26 21:29:34.261488: E tensorflow/stream_executor/cuda/cuda_driver.cc:397] failed call to cuInit: CUDA_ERROR_UNKNOWN
2018-12-26 21:29:34.261583: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:150] kernel driver does not appear to be running on this host (vm101): /proc/driver/nvidia/version does not exist
["Non Rust"]


In [3]:
!az ml computetarget attach aks -n aks -i /subscriptions/2134890c-7972-4f85-ad83-ac246ec2fbfe/resourcegroups/Container1/providers/Microsoft.ContainerService/managedClusters/aks -g Container1 -w workspace

Attaching compute resource...
Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '289', 'Content-Type': 'application/json', 'Expires': '-1', 'x-ms-request-id': '21b1cc19-92a6-409b-8b0d-1dbdd28597ec', 'x-ms-ratelimit-remaining-subscription-writes': '1199', 'Request-Context': 'appId=cid-v1:6a27ce65-5555-41a3-85f7-b7a1ce31fd6b', 'x-ms-client-request-id': '7997abe7-4ee4-4b9a-a3c7-233c5067d567', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-correlation-request-id': '21b1cc19-92a6-409b-8b0d-1dbdd28597ec', 'x-ms-routing-request-id': 'WESTINDIA:20181226T095930Z:21b1cc19-92a6-409b-8b0d-1dbdd28597ec', 'X-Content-Type-Options': 'nosniff', 'Date': 'Wed, 26 Dec 2018 09:59:30 GMT'}
Content: b'{"code":"BadRequest","message":"The request is invalid","error":{"code":"BadRequest","statusCode":400,"message":"The request is invalid","details":[{"code":"Inva

In [12]:
from azureml.core import Experiment, Run, Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.59


In [13]:
ws = Workspace(resource_group='Container1',workspace_name='workspace',subscription_id='2134890c-7972-4f85-ad83-ac246ec2fbfe')

In [14]:
from azureml.core.model import Model

In [15]:
model_name = "kerasmodel"
model_path = "./kerasmodel.pkl"

model = Model.register(
    model_path=model_path,
    model_name=model_name,
    tags={"data": "rust", "model": "classification"},
    description="Classifying rust and non rust images",
    workspace=ws
)

Registering model kerasmodel


In [16]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aks_name = 'aks'
aks_target = AksCompute(workspace=ws, name=aks_name)
print('found existing:', aks_target.name)


found existing: aks


In [25]:
import os
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("keras")

with open(os.path.join("myenv.yml"),"w") as f:
    f.write(myenv.serialize_to_string())

In [18]:
import os
with open(os.path.join("myenv.yml"),"r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
    - azureml-defaults
    - h5py
    - pillow
- keras



In [19]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2, 
    memory_gb=2, 
    tags={"data": "rust",  "method" : "keras"}, 
    description='Predict rust with keras'
)

In [20]:
%%time
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(
    execution_script="script.py", 
    runtime="python", 
    conda_file="myenv.yml"
)

image = ContainerImage.create(
    name = "keras-rust-img",
    models = [model],
    image_config = image_config,
    workspace = ws
)

image.wait_for_creation(show_output = True)

Creating image
Running.....................................................
SucceededImage creation operation finished for image keras-rust-img:13, operation "Succeeded"
CPU times: user 1.31 s, sys: 56.3 ms, total: 1.37 s
Wall time: 5min 19s


In [21]:
print(image.image_location)

workspace5846851499.azurecr.io/keras-rust-img:13


In [22]:
%%time
from azureml.core.webservice import Webservice

aci_service = Webservice.deploy_from_image(
    workspace = ws, 
    name = 'keras-rust8-aci-svc',
    image = image,
    deployment_config = aciconfig
)

aci_service.wait_for_deployment(show_output=True)

Creating service
Running..........................................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 921 ms, sys: 46.3 ms, total: 967 ms
Wall time: 4min 15s


In [23]:
print(aci_service.scoring_uri)

http://104.45.69.38:80/score


In [62]:
aci_service.get_logs()

'2018-12-26T14:58:44,260351809+00:00 - iot-server/run \n2018-12-26T14:58:44,260705209+00:00 - rsyslog/run \nok: run: gunicorn: (pid 13) 0s\nok: run: nginx: (pid 12) 0s\nok: run: rsyslog: (pid 14) 0s\n2018-12-26T14:58:44,263027607+00:00 - gunicorn/run \nok: run: rsyslog: (pid 14) 0s\n2018-12-26T14:58:44,266578304+00:00 - nginx/run \nok: run: rsyslog: (pid 14) 0s\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2018-12-26T14:58:44,331867849+00:00 - iot-server/finish 1 0\n2018-12-26T14:58:44,332934348+00:00 - Exit code 1 is normal. Not restarting iot-server.\n{"timestamp": "2018-12-26T14:58:44.571301Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-3d6a6752098d462eb73eeb5664770cb5-1970985ce317a833228788", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}\n{"timestamp": "